## Imports and Setup

In [1]:
import os
import pandas as pd
import geopandas as gpd
import logging
import joblib
import torch

import sys
sys.path.insert(0, "../src")
import sat_download

sys.path.insert(0, "../utils/")
import data_utils
import config_utils
import pred_utils
import embed_utils

%load_ext autoreload
%autoreload 2

## Load Config Files

In [19]:
iso_code = "BWA"
adm_level = "ADM2"
shapename = "Mahalapye"
spacing = 150
buffer_size = 150

cwd = os.path.dirname(os.getcwd())
data_config_file = os.path.join(cwd, "configs/data_config.yaml")
data_config = config_utils.load_config(data_config_file)

sat_config_file = os.path.join(cwd, "configs/sat_configs/sat_config_500x500_60cm.yaml")
sat_creds_file = os.path.join(cwd, "configs/sat_configs/sat_creds.yaml")

sat_config = config_utils.load_config(sat_config_file)
sat_creds = config_utils.create_config(sat_creds_file)

## Generate Prediction Tiles

In [20]:
tiles = pred_utils.generate_pred_tiles(
    data_config, iso_code, spacing, buffer_size, adm_level, shapename
).reset_index()
tiles["UID"] = list(tiles.index)

out_dir = data_utils._makedir(os.path.join(cwd, "output", iso_code, "tiles"))
out_file = os.path.join(out_dir, f"{iso_code}_{shapename}.gpkg")
tiles[["geometry"]].to_file(out_file, driver="GPKG")

print(f"Total tiles: {tiles.shape}")
tiles.head(3)

100%|████████████████████| 136/136 [03:25<00:00,  1.51s/it]                                                             


Total tiles: (32263, 12)


,index,geometry,shapeName,shapeISO,shapeID,shapeGroup,shapeType,points,index_right,type,properties,UID
0,1496156,"POLYGON ((3052469.260 -2622595.005, 3052469.26...",Mahalapye,,29308430B30543037727108,BWA,ADM2,POINT (3052319.260 -2622745.005),1037,Feature,"{'height': -1.0, 'confidence': -1.0}",0
1,1496157,"POLYGON ((3052469.260 -2622445.005, 3052469.26...",Mahalapye,,29308430B30543037727108,BWA,ADM2,POINT (3052319.260 -2622595.005),1037,Feature,"{'height': -1.0, 'confidence': -1.0}",1
2,1497176,"POLYGON ((3052619.260 -2622595.005, 3052619.26...",Mahalapye,,29308430B30543037727108,BWA,ADM2,POINT (3052469.260 -2622745.005),1037,Feature,"{'height': -1.0, 'confidence': -1.0}",2


## Download Satellite Images

In [22]:
data = tiles.copy()
data["geometry"] = data["points"]
sat_dir = os.path.join(cwd, "output", iso_code, "images", shapename)
sat_download.download_sat_images(sat_creds, sat_config, data=data, out_dir=sat_dir)

INFO:root:Data dimensions: (32263, 14), CRS: EPSG:3857
100%|████████████████████| 32263/32263 [00:01<00:00, 19564.83it/s]                                                      


## CNN Model

In [18]:
model_config_file = os.path.join(cwd, f"configs/cnn_configs/vgg16.yaml")
model_config = config_utils.load_config(model_config_file)
model_file = os.path.join(cwd, f"configs/cnn_configs/vgg16.yaml")

result = pred_utils.cnn_predict(
    tiles, iso_code, shapename, model_config, sat_dir, n_classes=2, model_file=model_file
)

INFO:root:Model file /home/itingzon.unicef/giga/exp/BWA_vgg16/BWA_vgg16.pth successfully loaded.
100%|████████████████████| 9417/9417 [25:15<00:00,  6.22it/s]                                                           


## ViT Model

In [27]:
model_config_file = os.path.join(cwd, f"configs/model_configs/dinov2_vitl14-SVC.yaml")
model_config = config_utils.load_config(model_config_file)

result = pred_utils.vit_pred(tiles, model_config, iso_code, shapename, sat_dir)

Using cache found in /home/itingzon.unicef/.cache/torch/hub/facebookresearch_dinov2_main
INFO:dinov2:using MLP layer as FFN
INFO:root:Device: cpu
INFO:root:Generating embeddings...
INFO:root:Reading file /home/itingzon.unicef/giga/output/BWA/embeddings/BWA_Mahalapye_dinov2_vitl14_embeds.csv
INFO:root:Loaded /home/itingzon.unicef/giga/exp/BWA-dinov2_vitl14-SVC/BWA-dinov2_vitl14-SVC.pkl
INFO:root:Generated /home/itingzon.unicef/giga/output/BWA/results/BWA_Mahalapye_dinov2_vitl14-SVC_results.gpkg
